由于 akshare.stock_profile_cninfo 公司概况-巨潮资讯 提供历史简称变更信息，所以这个文件不再需要执行了。留着供参考。  
Function: 调用 akshare.stock_info_change_name(symbol='000001') 接口，获取股票历史简称。需要处理的股票，交易所=上交所、北交所。  
支持QPS限制；支持断点续传。  

In [1]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

import random
import time
import akshare as ak
import pandas as pd

# 初始化change_name_sina，用于存储股票历史简称，包含两列：证券代码、历史简称
change_name_sina = pd.read_csv('./data/change_name_sina.csv')  # 每次启动，如果存在本地文件则先读取
# 如果本地文件不存在，则初始化一个空的DataFrame
if len(change_name_sina) == 0:
    change_name_sina = pd.DataFrame(columns=['证券代码', '历史简称']) # 初始化一个空的DataFrame

stock_info_df = pd.read_csv('data/stock_info.csv')  # 获取本地股票列表
stock_info_df['id'] = stock_info_df['证券完整代码'].str.split('.').str[0] # 临时新增一列'id'，为证券代码不含交易所后缀
change_name_sina_set = set(map(str, change_name_sina['证券代码'])) # 在set中查找元素的时间复杂度为O(1)，比list\array效率高。一定要将每个元素的数据类型转为字符串，否则无法和id正常匹配。

In [3]:
# 对stock_info_df中，所属市场=上交所、北交所 的逐个股票，调用 股票更名 接口，获取’历史简称‘。
for i in range(len(stock_info_df)):
    id = stock_info_df.loc[i, 'id']
    # 排除深交所股票
    
    if  stock_info_df.loc[i, '所属市场'] == '上交所' or stock_info_df.loc[i, '所属市场'] == '北交所':
        if  id not in change_name_sina_set: # 排除之前已经获取并写入到change_name_sina的股票
            # 调用 股票更名 接口，获取 ’历史简称‘ List。如果接口调用失败，是因为有QPS限制，随机暂停n秒，然后再等待重试。
            while True: # 一直循环，直到接口调用成功。
                try: # 尝试调用接口
                    n_list = ak.stock_info_change_name(symbol=id) # ’历史简称‘ List
                    break # 如果调用成功，则跳出while循环
                except: # 如果引发了异常
                    print('接口调用失败，等待n秒后重试。')
                    time.sleep(random.randint(30, 100))
            # 如果n_list为空，则表示该股票没有历史简称。所以仅当n_list不为空时，才将结果写入change_name_sina。
            if len(n_list) != 0:
                n_str = ','.join(n_list['name'])          # 将n_list转为逗号分隔的字符串n_str。
                print('已获取：'+id,n_str)
                change_name_sina = change_name_sina._append({'证券代码': id, '历史简称': n_str}, ignore_index=True) # 将结果写入change_name_sina
                # i每循环50次，将结果写入本地文件。
                if i % 50 == 0:
                    change_name_sina.to_csv('data/change_name_sina.csv', index=False, encoding='utf-8-sig') # 保存为utf-8格式，避免乱码。
                    print('已写入本地文件。')              
            time.sleep(random.randint(5, 15)) # 每次调用接口后，随机暂停n秒，避免QPS限制。
print('获取完毕')
# 测试结论：通过。